In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:
sent = pd.read_csv('Hotel_Reviews.csv')
# sent.head()
analyzer = SentimentIntensityAnalyzer()
def compound_score(txt):
    return analyzer.polarity_scores(txt)["compound"]



In [7]:
def sentiment(score):
    emotion = 0
    if score >= 0.5:
        emotion = 2
    elif score <= -0.5:
        emotion = 0
    else:
        emotion = 1
    return emotion

In [8]:
positive = sent['Positive_Review']
negative = sent['Negative_Review']
frames = [positive,negative]
reviews = pd.concat(frames)
reviews  = reviews.sample(frac=1)
polarity_scores = reviews.astype("str").apply(compound_score)
reviewls = list(reviews)
final = pd.DataFrame(list(zip(reviewls,polarity_scores)))
final.columns = ['Reviews','Sentiment'] 

KeyboardInterrupt: 

In [5]:
final["Sentiment"] = final["Sentiment"].apply(sentiment)

In [10]:
final = pd.read_csv('hrwithsentiment1.csv')
final.head()

,Reviews,Sentiment
0,Location,1
1,Location is a bit far and sometimes reception...,1
2,A wonderful receptionist Aneta Debosz who cou...,2
3,Indifferent staff Wouldn t speak unless spoke...,2
4,Daily breakfast not much choices and over 4 n...,1


In [3]:
nlp = spacy.load('en_core_web_lg')

In [3]:
# with nlp.disable_pipes():
#     vectors = np.array([nlp(review.Reviews).vector for idx, review in final.iterrows()])
    
# vectors.shape
# np.save('vectors.npy',vectors)
a=np.load('vectors.npy')
a.shape

(1031476, 300)

In [13]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import pickle

X_train, X_test, y_train, y_test = train_test_split(vectors, final.Sentiment, test_size=0.1, random_state=1)

model = LinearSVC(random_state=1, dual=False)

model.fit(X_train, y_train)

print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')

Model test accuracy: 88.350%


In [14]:
pickle.dump(model, open('hrclassificationneutral.sav','wb'))

In [20]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv', index_col = False)
df = df[:1000]
df.drop(columns=['Rating'], inplace=True)
df.head()

,Review
0,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...
3,"unique, great stay, wonderful time hotel monac..."
4,"great stay great stay, went seahawk game aweso..."


In [25]:
with nlp.disable_pipes():
    testvectors = np.array([nlp(Review).vector for Review in df['Review']])
testvectors.shape

(1000, 300)

In [26]:
def senanalyze(x):
    y = []
    for i in x:
        if(i == 0):
            y.append("Negative")
        if(i == 1):
            y.append("Neutral")
        if(i == 2):
            y.append("Positive")
    return y

In [27]:
ls = []
# loaded = pickle.load(open('hrclassificationneutral.sav','rb'))
ls = model.predict(testvectors)

In [28]:
n = senanalyze(ls)

In [31]:
df['Sen'] = n
df.head
df.to_csv('resul1.csv')